In [ ]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib as mpl

plt.rcParams['axes.unicode_minus'] = False

path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'
font_name = mpl.font_manager.FontProperties(fname=path).get_name()

mpl.rc('font', family=font_name)

mpl.font_manager._rebuild()

In [ ]:
plt.plot([0,1], [0,1])
plt.title('한글 연습')
plt.show()

In [ ]:
import numpy as np
import pandas as pd

crime_raw_data = pd.read_csv('2018년.csv', encoding = 'euc-kr')
crime_raw_data.head()

In [ ]:
crime_raw_data.info()

In [ ]:
crime_raw_data['죄종'].unique()

In [ ]:
crime_raw_data = crime_raw_data[crime_raw_data['죄종'].notnull()]
crime_raw_data.info()

In [ ]:
crime_station = crime_raw_data.pivot_table(
    crime_raw_data, index = ['구분'], columns = ['죄종', '발생검거'], aggfunc = np.sum
)

crime_station.head()

In [ ]:
crime_station.columns

In [ ]:
crime_station.columns = crime_station.columns.droplevel([0])
crime_station.columns

In [ ]:
crime_station['강도', '검거']

In [ ]:
crime_station['살인', '발생']

In [ ]:
crime_station.loc['강남', ('강도', '발생')]

In [ ]:
crime_station.head()

In [ ]:
tmp = crime_station.columns.get_level_values(0) + \
crime_station.columns.get_level_values(1)

tmp

In [ ]:
crime_station.columns = tmp
crime_station.head()

In [ ]:
crime_station.info()

In [ ]:
real_gu = {'종로구' : ['종로', '혜화'], '중구' : ['남대문', '중부'],
           '용산구' : ['용산'], '성동구' : ['성동'],
           '광진구' : ['광진'], '동대문구' : ['동대문'],
           '중랑구' : ['중랑'], '성북구' : ['성북', '종암'],
           '강북구' : ['강북'], '도봉구' : ['도봉'],
           '노원구' : ['노원'], '은평구' : ['서부', '은평'],
           '서대문구' : ['서대문'], '마포구' : ['마포'],
           '양천구' : ['양천'], '강서구' : ['강서'],
           '구로구' : ['구로'], '금천구' : ['금천'],
           '영등포구' : ['영등포'], '동작구' : ['동작'],
           '관악구' : ['관악'], '서초구' : ['방배', '서초'],
           '강남구' : ['강남', '수서'], '송파구' : ['송파'],
           '강동구' : ['강동']}

In [ ]:
for gu_name, station_name in real_gu.items():
    if '강남' in station_name:
        print(gu_name)

In [ ]:
tmp_gu = []

for station in crime_station.index:
    for gu_name, station_name in real_gu.items():
        if station in station_name:
            tmp_gu.append(gu_name)

tmp_gu

In [ ]:
crime_station['구'] = tmp_gu
crime_station

In [ ]:
crime_gu = pd.pivot_table(crime_station, index = '구', aggfunc = np.sum)
crime_gu.head()

In [ ]:
crime_gu['살인발생'].sort_values(ascending = False).head()

In [ ]:
crime_gu['폭력발생'].sort_values(ascending = False).head()

In [ ]:
crime_gu['강도발생'].sort_values(ascending = False).head()

In [ ]:
crime_gu['절도발생'].sort_values(ascending = False).head()

In [ ]:
crime_gu['강간발생'].sort_values(ascending = False).head()

In [ ]:
crime_gu['강도검거']/crime_gu['강도발생']

In [ ]:
num = ['강간검거', '강도검거', '살인검거', '절도검거', '폭력검거']
den = ['강간발생', '강도발생', '살인발생', '절도발생', '폭력발생']

crime_gu[num].div(crime_gu[den].values)

In [ ]:
target = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']

crime_gu[target] = crime_gu[num].div(crime_gu[den].values) * 100
crime_gu.head()

In [ ]:
crime_gu = crime_gu.drop(columns = num)
crime_gu.head()

In [ ]:
crime_gu[target] > 100

In [ ]:
crime_gu[crime_gu[target] > 100] = 100
crime_gu

In [ ]:
crime_gu.rename(columns = {'강간발생' : '강간',
                           '강도발생' : '강도',
                           '살인발생' : '살인',
                           '절도발생' : '절도',
                           '폭력발생' : '폭력'},
                inplace = True)
crime_gu.head()

In [ ]:
col = ['살인', '강도', '강간', '절도', '폭력']
crime_gu_norm = crime_gu[col] / crime_gu[col].max()
crime_gu_norm.head()

In [ ]:
col2 = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']
crime_gu_norm[col2] = crime_gu[col2]
crime_gu_norm.head()

In [ ]:
crime_gu_norm['범죄'] = np.mean(crime_gu_norm[col], axis = 1)
crime_gu_norm.head()

In [ ]:
crime_gu_norm['범죄'].sort_values(ascending = False).head()

In [ ]:
crime_gu_norm['검거'] = np.mean(crime_gu_norm[col2], axis = 1)
crime_gu_norm.head()

In [ ]:
crime_gu_norm['검거'].sort_values(ascending = False).head()

In [ ]:
crime_gu_norm['검거'].sort_values().head()

In [ ]:
import seaborn as sns

target_col = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율', '검거']

crime_gu_norm_sort = crime_gu_norm.sort_values(by = '검거', ascending = False)

plt.figure(figsize = (10, 10))
sns.heatmap(crime_gu_norm_sort[target_col],
            annot = True, fmt = 'f', linewidth = .5, cmap = 'RdPu')
plt.title('범죄 검거 비율 (정규화된 검거의 합으로 정렬)')
plt.show()

In [ ]:
target_col = ['강간', '강도', '살인', '절도', '폭력', '범죄']

crime_gu_norm_sort = crime_gu_norm.sort_values(by = '범죄', ascending = False)

plt.figure(figsize = (10, 10))
sns.heatmap(crime_gu_norm_sort[target_col],
            annot = True, fmt = 'f', linewidth = .5, cmap = 'RdPu')
plt.title('범죄 비율 (정규화된 발생 건수로 정렬)')
plt.show()

In [ ]:
import folium

m = folium.Map(location=[45.5236, -122.6750])
m

In [ ]:
folium.Map(location = [22.20111, 113.54417], tiles = 'Stamen Toner', zoom_start = 13)

In [ ]:
folium.Map(location = [22.20111, 113.54417], tiles = 'Stamen Terrain', zoom_start = 13)

In [ ]:
folium.Map(location = [22.20111, 113.54417], tiles = 'Stamen Watercolor', zoom_start = 13)

In [ ]:
my_map = folium.Map(location = [45.372, -121.6972], zoom_start = 12,
                    tiles = 'Stamen Terrain')

folium.Marker([45.3288, -121.6625],
              popup = '<i>Mt. Hood Meadows</i>').add_to(my_map)
folium.Marker([45.3311, -121.7113],
              popup = '<b>Timberline Lodge</b>').add_to(my_map)

my_map

In [ ]:
m = folium.Map(location = [45.5236, -122.6750], tiles = 'Stamen Toner',
               zoom_start = 13)

folium.CircleMarker(location = [45.5215, -122.6261], radius=50,
                    popup = 'Laurelhurst Park', color = '#3186cc',
                    fill = True, fill_color = '#3186cc').add_to(m)

m

In [ ]:
import json


geo_str = json.load(open('./skorea_municipalities_geo_simple.json', encoding = 'utf-8'))
geo_str

In [ ]:
crime_gu.head()

In [ ]:
crime_gu.reset_index().rename(columns = {"index": "구"})
crime_gu.rename_axis('구').reset_index()
crime_gu.index.name = '구'
crime_gu = crime_gu.reset_index()
crime_gu.head()

In [ ]:
my_map = folium.Map(location = [37.5502, 126.982], zoom_start = 12,
                    tiles = 'Stamen Toner')

folium.Choropleth(
    geo_data = geo_str,
    data = crime_gu, columns = ['구','살인'],
    nan_fill_color = 'purple', nan_fill_opacity = 0.4,
    key_on = 'feature.id', fill_color = 'PuRd',
    legend_name = 'Number of murder cases'
).add_to(my_map)

my_map

In [ ]:
my_map = folium.Map(location = [37.5502, 126.982], zoom_start = 12,
                    tiles = 'Stamen Toner')

folium.Choropleth(
    geo_data = geo_str,
    data = crime_gu, columns = ['구','절도'],
    nan_fill_color = 'purple', nan_fill_opacity = 0.4,
    key_on = 'feature.id', fill_color = 'PuRd',
    legend_name = 'Number of theft cases'
).add_to(my_map)

my_map

In [ ]:
my_map = folium.Map(location = [37.5502, 126.982], zoom_start = 12,
                    tiles = 'Stamen Toner')

folium.Choropleth(
    geo_data = geo_str,
    data = crime_gu, columns = ['구','폭력'],
    nan_fill_color = 'purple', nan_fill_opacity = 0.4,
    key_on = 'feature.id', fill_color = 'PuRd',
    legend_name = 'Number of violence cases'
).add_to(my_map)

my_map